**Modélisation prédictive avec le jeu de données Titanic (Kaggle Beginners)**

Ce projet vise à initier les apprenants à la résolution d’un problème réel de classification supervisée à partir de données ouvertes. À partir du jeu de données Titanic (projet débutant de Kaggle), ils construisent un modèle prédictif pour estimer la probabilité de survie des passagers, en mobilisant les étapes clés d’un pipeline de machine learning.